# Comparison of Kfold Implementations

## Import Statements

In [2]:
from cold_crossval import *

In [25]:
import numpy as np
import pandas as pd
import time
import timeit
import line_profiler
import memory_profiler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error

## Data Loading

In [37]:
overall_poke = data_reduction("../pokedex.csv", ["status"])[["name", "status", "type_number", 
        "height_m", "weight_kg", "abilities_number", 
        "total_points", "hp", "attack", 
        "defense", "sp_attack", "sp_defense", 
        "speed", "egg_type_number"]]
overall_poke_np = overall_poke.dropna(axis = 0).to_numpy()

In [5]:
%load_ext memory_profiler

## Cold Implementation

In [29]:
%%timeit -r7 -n15
cold_kfoldcv(data = overall_poke_np,
         col_names = ["name", "status", "type_number", "height_m", "weight_kg", "abilities_number", "total_points", "hp", "attack", "defense", "sp_attack", "sp_defense", "speed", "egg_type_number"], 
         inputs = ["total_points", "hp", "attack", "defense", "sp_attack", "sp_defense", "speed"], 
         output = "status", 
         k = 10)

17.1 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [7]:
%%time
cold_kfoldcv(data = overall_poke_np,
         col_names = ["name", "status", "type_number", "height_m", "weight_kg", "abilities_number", "total_points", "hp", "attack", "defense", "sp_attack", "sp_defense", "speed", "egg_type_number"], 
         inputs = ["total_points", "hp", "attack", "defense", "sp_attack", "sp_defense", "speed"], 
         output = "status", 
         k = 10)

CPU times: total: 31.2 ms
Wall time: 19.8 ms


17.547991623834

In [30]:
%memit cold_kfoldcv(data = overall_poke_np, col_names = ["name", "status", "type_number", "height_m", "weight_kg", "abilities_number", "total_points", "hp", "attack", "defense", "sp_attack", "sp_defense", "speed", "egg_type_number"], inputs = ["total_points", "hp", "attack", "defense", "sp_attack", "sp_defense", "speed"], output = "status", k = 10)

peak memory: 145.27 MiB, increment: 0.00 MiB


In [19]:
train, test = train_test_split(overall_poke_np, train_size = .9, random_state = 2022)

tree = DecisionTreeClassifier(ccp_alpha = 0.001, max_depth = 3)
tree.fit(train[:, 6:13], train[:, 1].astype(int))

preds = tree.predict(test[:, 6:13])

cold_classification(preds, test[:, 1])

0.02912621359223301

## Sklearn Implementation

In [32]:
%%timeit -r7 -n15

folds = KFold(n_splits = 10)

tree = DecisionTreeClassifier(ccp_alpha = 0.001, max_depth = 3)
cv_result = cross_val_score(tree, overall_poke_np[:, 6:13], overall_poke_np[:, 1].astype(int), cv = 10, scoring = 'accuracy')

mean_cv = np.mean(cv_result)
# print(1 - mean_cv)

18.6 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 15 loops each)


In [12]:
%%time

folds = KFold(n_splits = 10)

scorer = make_scorer(score_func = accuracy_score)
tree = DecisionTreeClassifier(ccp_alpha = 0.001, max_depth = 3)
cv_result = cross_val_score(tree, overall_poke_np[:, 6:13], overall_poke_np[:, 1].astype(int), cv = 10, scoring = make_scorer(accuracy_score))

mean_cv = np.mean(cv_result)

CPU times: total: 15.6 ms
Wall time: 23.2 ms


In [33]:
def fold_func(data):
    folds = KFold(n_splits = 10)
    tree = DecisionTreeClassifier(ccp_alpha = 0.001, max_depth = 3)

    cv_result = cross_val_score(tree, data[:, 6:13], data[:, 1].astype(int), cv = 10, scoring = "accuracy")
    mean_cv = np.mean(cv_result)

In [34]:
%memit fold_func(overall_poke_np)

peak memory: 145.43 MiB, increment: 0.00 MiB
